## 시맨틱 청킹 소개
텍스트 청킹은 검색 증강 생성(RAG)에서 필수적인 단계로, 큰 텍스트 본문을 의미 있는 세그먼트로 나누어 검색 정확도를 향상시킵니다.
고정 길이 청킹과 달리, 시맨틱 청킹은 문장 간의 내용 유사성을 기반으로 텍스트를 분할합니다. 즉, 의미적으로 유사한 문장들을 하나의 청크로 묶고, 의미가 크게 달라지는 지점에서 청크를 나눕니다. 이는 문맥을 더 잘 보존하고 관련성 높은 정보를 검색하는 데 도움이 됩니다.

### 분할 지점 결정 방법 (Breakpoint Methods):
- **백분위수 (Percentile)**: 모든 유사도 차이의 X번째 백분위수를 찾아, 그 값보다 유사도 하락이 큰 지점에서 청크를 분할합니다. 예를 들어, 90번째 백분위수를 기준으로 하면, 유사도 하락폭이 상위 10%에 해당하는 지점에서 분할합니다.
- **표준 편차 (Standard Deviation)**: 유사도가 평균보다 X 표준 편차 이상으로 떨어지는 지점에서 분할합니다. 이는 통계적으로 유의미한 유사도 변화를 감지하는 방법입니다.
- **사분위수 범위 (Interquartile Range, IQR)**: 사분위수 범위(Q3 - Q1)를 사용하여 분할 지점을 결정합니다. IQR은 데이터의 변동성을 측정하는 지표로, 이상치(outlier)를 감지하는 데 사용될 수 있으며, 여기서는 급격한 유사도 변화 지점을 찾는 데 응용됩니다.

이 노트북은 **백분위수 방법을 사용한** 시맨틱 청킹을 구현하고 샘플 텍스트에 대한 성능을 평가합니다.

## 환경 설정
필요한 라이브러리를 가져오는 것으로 시작합니다.

In [1]:
import fitz # PyMuPDF 라이브러리
import os
import numpy as np
import json
from openai import OpenAI

## PDF 파일에서 텍스트 추출
RAG를 구현하려면 먼저 텍스트 데이터 소스가 필요합니다. 여기서는 PyMuPDF 라이브러리를 사용하여 PDF 파일에서 텍스트를 추출합니다.

In [2]:
def extract_text_from_pdf(pdf_path):
    """
    PDF 파일에서 텍스트를 추출합니다.

    Args:
    pdf_path (str): PDF 파일 경로.

    Returns:
    str: PDF에서 추출된 텍스트.
    """
    # PDF 파일 열기
    mypdf = fitz.open(pdf_path)
    all_text = ""  # 추출된 텍스트를 저장할 빈 문자열 초기화
    
    # PDF의 각 페이지 반복
    for page in mypdf:
        # 현재 페이지에서 텍스트를 추출하고 공백 추가
        all_text += page.get_text("text") + " "

    # 앞뒤 공백을 제거한 추출된 텍스트 반환
    return all_text.strip()

# PDF 파일 경로 정의
pdf_path = "data/AI_Information.pdf"

# PDF 파일에서 텍스트 추출
extracted_text = extract_text_from_pdf(pdf_path)

# 추출된 텍스트의 처음 500자 출력
print(extracted_text[:500])

Understanding Artificial Intelligence 
Chapter 1: Introduction to Artificial Intelligence 
Artificial intelligence (AI) refers to the ability of a digital computer or computer-controlled robot 
to perform tasks commonly associated with intelligent beings. The term is frequently applied to 
the project of developing systems endowed with the intellectual processes characteristic of 
humans, such as the ability to reason, discover meaning, generalize, or learn from past 
experience. Over the past f


## OpenAI API 클라이언트 설정
임베딩과 응답을 생성하기 위해 OpenAI 클라이언트를 초기화합니다.

In [ ]:
# 기본 URL과 API 키로 OpenAI 클라이언트 초기화
client = OpenAI(
    base_url="https://api.studio.nebius.com/v1/",
    api_key=os.getenv("OPENAI_API_KEY")  # 환경 변수에서 API 키 검색
)

## 문장 수준 임베딩 생성
텍스트를 문장으로 분할하고 임베딩을 생성합니다.
시맨틱 청킹을 위해서는 먼저 텍스트를 개별 문장으로 나누어야 합니다. 그런 다음 각 문장의 의미를 나타내는 벡터 표현, 즉 임베딩을 생성합니다. 이 임베딩 벡터 간의 유사도를 비교하여 의미적 유사성을 측정할 수 있습니다.

In [4]:
def get_embedding(text, model="BAAI/bge-en-icl"):
    """
    OpenAI를 사용하여 주어진 텍스트에 대한 임베딩을 생성합니다.

    Args:
    text (str): 입력 텍스트.
    model (str): 임베딩 모델 이름.

    Returns:
    np.ndarray: 임베딩 벡터.
    """
    response = client.embeddings.create(model=model, input=text)
    return np.array(response.data[0].embedding)

# 텍스트를 문장으로 분할 (기본 분할)
sentences = extracted_text.split(". ") # 마침표와 공백을 기준으로 문장을 나눕니다. 더 정교한 문장 분리 라이브러리(예: NLTK, spaCy)를 사용할 수도 있습니다.

# 각 문장에 대한 임베딩 생성
embeddings = [get_embedding(sentence) for sentence in sentences]

print(f"생성된 문장 임베딩 수: {len(embeddings)}")

Generated 257 sentence embeddings.


## 유사도 차이 계산
연속된 문장 간의 코사인 유사도를 계산합니다.
인접한 두 문장의 임베딩 벡터 간 코사인 유사도를 계산하여, 두 문장이 의미적으로 얼마나 유사한지를 측정합니다. 유사도 값이 높을수록 두 문장은 의미적으로 가깝고, 낮을수록 멀다고 할 수 있습니다.

In [5]:
def cosine_similarity(vec1, vec2):
    """
    두 벡터 간의 코사인 유사도를 계산합니다.

    Args:
    vec1 (np.ndarray): 첫 번째 벡터.
    vec2 (np.ndarray): 두 번째 벡터.

    Returns:
    float: 코사인 유사도.
    """
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

# 연속된 문장 간의 유사도 계산
similarities = [cosine_similarity(embeddings[i], embeddings[i + 1]) for i in range(len(embeddings) - 1)]

## 시맨틱 청킹 구현
분할 지점을 찾기 위한 세 가지 다른 방법을 구현합니다.
계산된 연속 문장 간 유사도 값들을 사용하여, 어디서 청크를 나눌지(분할 지점) 결정합니다. 유사도가 급격히 떨어지는 지점이 의미적으로 다른 내용으로 넘어가는 부분일 가능성이 높습니다.

In [6]:
def compute_breakpoints(similarities, method="percentile", threshold=90):
    """
    유사도 하락을 기반으로 청킹 분할 지점을 계산합니다.

    Args:
    similarities (List[float]): 문장 간 유사도 점수 목록.
    method (str): 'percentile', 'standard_deviation' 또는 'interquartile'.
    threshold (float): 임계값 (백분위수의 경우 'percentile', 표준 편차의 경우 'standard_deviation').

    Returns:
    List[int]: 청크 분할이 발생해야 하는 인덱스 목록.
    """
    # 선택한 방법에 따라 임계값 결정
    if method == "percentile":
        # 유사도 점수의 X번째 백분위수 계산
        threshold_value = np.percentile(similarities, threshold)
    elif method == "standard_deviation":
        # 유사도 점수의 평균 및 표준 편차 계산
        mean = np.mean(similarities)
        std_dev = np.std(similarities)
        # 임계값을 평균 - (X * 표준 편차)로 설정
        threshold_value = mean - (threshold * std_dev)
    elif method == "interquartile":
        # 1사분위수(Q1)와 3사분위수(Q3) 계산
        q1, q3 = np.percentile(similarities, [25, 75])
        # IQR 규칙을 사용하여 이상치에 대한 임계값 설정
        threshold_value = q1 - 1.5 * (q3 - q1)
    else:
        # 잘못된 방법이 제공되면 오류 발생
        raise ValueError("잘못된 방법입니다. 'percentile', 'standard_deviation' 또는 'interquartile' 중에서 선택하십시오.")

    # 유사도가 임계값 아래로 떨어지는 인덱스 식별
    return [i for i, sim in enumerate(similarities) if sim < threshold_value]

# 백분위수 방법을 사용하여 임계값 90으로 분할 지점 계산
breakpoints = compute_breakpoints(similarities, method="percentile", threshold=90)

## 텍스트를 시맨틱 청크로 분할
계산된 분할 지점을 기반으로 텍스트를 분할합니다.

In [7]:
def split_into_chunks(sentences, breakpoints):
    """
    문장을 시맨틱 청크로 분할합니다.

    Args:
    sentences (List[str]): 문장 목록.
    breakpoints (List[int]): 청킹이 발생해야 하는 인덱스 목록.

    Returns:
    List[str]: 텍스트 청크 목록.
    """
    chunks = []  # 청크를 저장할 빈 리스트 초기화
    start = 0  # 시작 인덱스 초기화

    # 각 분할 지점을 반복하여 청크 생성
    for bp in breakpoints:
        # 시작부터 현재 분할 지점까지의 문장 청크 추가
        chunks.append(". ".join(sentences[start:bp + 1]) + ".") # 나눠진 문장들을 다시 합치고 마침표를 붙여줍니다.
        start = bp + 1  # 시작 인덱스를 분할 지점 다음 문장으로 업데이트

    # 나머지 문장을 마지막 청크로 추가
    chunks.append(". ".join(sentences[start:]))
    return chunks  # 청크 목록 반환

# split_into_chunks 함수를 사용하여 청크 생성
text_chunks = split_into_chunks(sentences, breakpoints)

# 생성된 청크 수 출력
print(f"시맨틱 청크 수: {len(text_chunks)}")

# 결과를 확인하기 위해 첫 번째 청크 출력
print("\n첫 번째 텍스트 청크:")
print(text_chunks[0])

Number of semantic chunks: 231

First text chunk:
Understanding Artificial Intelligence 
Chapter 1: Introduction to Artificial Intelligence 
Artificial intelligence (AI) refers to the ability of a digital computer or computer-controlled robot 
to perform tasks commonly associated with intelligent beings.


## 시맨틱 청크에 대한 임베딩 생성
나중에 검색을 위해 각 청크에 대한 임베딩을 생성합니다.
이렇게 생성된 시맨틱 청크들은 이제 각각 하나의 단위로 취급되어 임베딩됩니다. 이 청크 임베딩들이 검색 대상이 됩니다.

In [8]:
def create_embeddings(text_chunks):
    """
    각 텍스트 청크에 대한 임베딩을 생성합니다.

    Args:
    text_chunks (List[str]): 텍스트 청크 목록.

    Returns:
    List[np.ndarray]: 임베딩 벡터 목록.
    """
    # get_embedding 함수를 사용하여 각 텍스트 청크에 대한 임베딩 생성
    return [get_embedding(chunk) for chunk in text_chunks]

# create_embeddings 함수를 사용하여 청크 임베딩 생성
chunk_embeddings = create_embeddings(text_chunks)

## 시맨틱 검색 수행
가장 관련성 높은 청크를 검색하기 위해 코사인 유사도를 구현합니다.

In [9]:
def semantic_search(query, text_chunks, chunk_embeddings, k=5):
    """
    질의에 가장 관련성 높은 텍스트 청크를 찾습니다.

    Args:
    query (str): 검색 질의.
    text_chunks (List[str]): 텍스트 청크 목록.
    chunk_embeddings (List[np.ndarray]): 청크 임베딩 목록.
    k (int): 반환할 상위 결과 수.

    Returns:
    List[str]: 상위 k개의 관련 청크.
    """
    # 질의에 대한 임베딩 생성
    query_embedding = get_embedding(query)
    
    # 질의 임베딩과 각 청크 임베딩 간의 코사인 유사도 계산
    similarities = [cosine_similarity(query_embedding, emb) for emb in chunk_embeddings]
    
    # 상위 k개의 가장 유사한 청크의 인덱스 가져오기
    top_indices = np.argsort(similarities)[-k:][::-1] # argsort는 오름차순으로 정렬된 인덱스를 반환하므로, 뒤에서 k개를 가져오고 순서를 뒤집습니다.
    
    # 상위 k개의 가장 관련성 높은 텍스트 청크 반환
    return [text_chunks[i] for i in top_indices]

In [10]:
# JSON 파일에서 검증 데이터 로드
with open('data/val.json') as f:
    data = json.load(f)

# 검증 데이터에서 첫 번째 질의 추출
query = data[0]['question']

# 상위 2개의 관련 청크 가져오기
top_chunks = semantic_search(query, text_chunks, chunk_embeddings, k=2)

# 질의 출력
print(f"질의: {query}")

# 상위 2개의 가장 관련성 높은 텍스트 청크 출력
for i, chunk in enumerate(top_chunks):
    print(f"문맥 {i+1}:\n{chunk}\n{'='*40}")

Query: What is 'Explainable AI' and why is it considered important?
Context 1:

Explainable AI (XAI) 
Explainable AI (XAI) aims to make AI systems more transparent and understandable. Research in 
XAI focuses on developing methods for explaining AI decisions, enhancing trust, and improving 
accountability.
Context 2:

Transparency and Explainability 
Transparency and explainability are essential for building trust in AI systems. Explainable AI (XAI) 
techniques aim to make AI decisions more understandable, enabling users to assess their 
fairness and accuracy.


## 검색된 청크를 기반으로 응답 생성

In [11]:
# AI 어시스턴트에 대한 시스템 프롬프트 정의
system_prompt = "당신은 주어진 문맥에 기반하여 엄격하게 답변하는 AI 어시스턴트입니다. 제공된 문맥에서 직접적으로 답변을 도출할 수 없는 경우, '답변하기에 충분한 정보가 없습니다.'라고 응답하십시오."

def generate_response(system_prompt, user_message, model="meta-llama/Llama-3.2-3B-Instruct"):
    """
    시스템 프롬프트와 사용자 메시지를 기반으로 AI 모델로부터 응답을 생성합니다.

    Args:
    system_prompt (str): AI의 행동을 안내하는 시스템 프롬프트.
    user_message (str): 사용자의 메시지 또는 질의.
    model (str): 응답 생성에 사용할 모델. 기본값은 "meta-llama/Llama-3.2-3B-Instruct"입니다. (원문: "meta-llama/Llama-2-7B-chat-hf")

    Returns:
    dict: AI 모델의 응답.
    """
    response = client.chat.completions.create(
        model=model,
        temperature=0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_message}
        ]
    )
    return response

# 상위 청크를 기반으로 사용자 프롬프트 생성
user_prompt = "\n".join([f"문맥 {i + 1}:\n{chunk}\n=====================================\n" for i, chunk in enumerate(top_chunks)])
user_prompt = f"{user_prompt}\n질문: {query}"

# AI 응답 생성
ai_response = generate_response(system_prompt, user_prompt)

## AI 응답 평가
AI 응답을 예상 답변과 비교하고 점수를 매깁니다.

In [12]:
# 평가 시스템에 대한 시스템 프롬프트 정의
evaluate_system_prompt = "당신은 AI 어시스턴트의 응답을 평가하는 지능형 평가 시스템입니다. AI 어시스턴트의 응답이 실제 응답과 매우 유사하면 1점을 부여하십시오. 응답이 실제 응답과 비교하여 부정확하거나 만족스럽지 않으면 0점을 부여하십시오. 응답이 실제 응답과 부분적으로 일치하면 0.5점을 부여하십시오."

# 사용자 질의, AI 응답, 실제 응답 및 평가 시스템 프롬프트를 결합하여 평가 프롬프트 생성
evaluation_prompt = f"사용자 질의: {query}\nAI 응답:\n{ai_response.choices[0].message.content}\n실제 응답: {data[0]['ideal_answer']}\n{evaluate_system_prompt}"

# 평가 시스템 프롬프트와 평가 프롬프트를 사용하여 평가 응답 생성
evaluation_response = generate_response(evaluate_system_prompt, evaluation_prompt)

# 평가 응답 출력
print(evaluation_response.choices[0].message.content)

Based on the evaluation criteria, I would assign a score of 0.5 to the AI assistant's response.

The response is partially aligned with the true response, as it correctly identifies the main goal of Explainable AI (XAI) as making AI systems more transparent and understandable. However, it lacks some key details and nuances present in the true response. For example, the true response mentions the importance of assessing fairness and accuracy, which is not explicitly mentioned in the AI assistant's response. Additionally, the true response uses more precise language, such as "providing insights into how they make decisions," which is not present in the AI assistant's response.
